In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def weight_variable(name, shape):
    return tf.get_variable(name, shape, initializer = tf.contrib.layers.xavier_initializer(seed = 1))

def bias_variable(name, shape):
    return tf.get_variable(name, shape, initializer = tf.zeros_initializer())

## Fully Connected Neural Nets

### Basic One-Layer NN

In [3]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

w1 = weight_variable('w1', [784,10])
b1 = bias_variable('b1', [10])

z1 = tf.matmul(x, w1) + b1

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = z1))
train_step = tf.train.AdamOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(z1,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(1000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))

step 0, train accuracy 0.14
step 100, train accuracy 0.78
step 200, train accuracy 0.9
step 300, train accuracy 0.88
step 400, train accuracy 0.84
step 500, train accuracy 0.91
step 600, train accuracy 0.83
step 700, train accuracy 0.9
step 800, train accuracy 0.81
step 900, train accuracy 0.95
test accuracy 0.8861


### Basic Two-Layer NN

In [4]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

w1 = weight_variable('w1', [784,20])
b1 = bias_variable('b1', [20])

z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.relu(z1)

w2 = weight_variable('w2', [20,10])
b2 = bias_variable('b2', [10])

z2 = tf.matmul(a1, w2) + b2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = z2))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(z2,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(1000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))

step 0, train accuracy 0.1
step 100, train accuracy 0.82
step 200, train accuracy 0.88
step 300, train accuracy 0.91
step 400, train accuracy 0.93
step 500, train accuracy 0.92
step 600, train accuracy 0.81
step 700, train accuracy 0.92
step 800, train accuracy 0.91
step 900, train accuracy 0.93
test accuracy 0.9306


### Basic Two-Layer NN using tf.contrib.learn

In [5]:
tf.reset_default_graph()
# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=784)]

classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                              hidden_units=[20, 10],
                                              n_classes=10,
                                              optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
                                              activation_fn = tf.nn.relu,
                                              model_dir="tmp/mnist_model")

def generate_input_fn(data, label):	
    image_batch, label_batch = tf.train.shuffle_batch(
            [data, label]
            , batch_size=100
            , capacity=800
            , min_after_dequeue=400
            , enqueue_many=True)
    return image_batch, label_batch

def input_fn_for_train():
    train_data = tf.constant(np.array(mnist.train.images, 'float32'))
    train_target = tf.argmax(tf.constant(np.array(mnist.train.labels, 'int64')), axis=1)
    return generate_input_fn(train_data, train_target)

# Fit model.
classifier.fit(input_fn=input_fn_for_train, steps=1000)

# Define the test inputs
def get_test_inputs():
    x = tf.constant(np.array(mnist.test.images, 'float32'))
    y = tf.argmax(tf.constant(np.array(mnist.test.labels, 'int64')), axis=1)

    return x, y

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                       steps=1)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_task_type': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_task_id': 0, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5b526c0b8>, '_keep_checkpoint_every_n_hours': 10000, '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_evaluation_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1001 into tmp/mnist_model

### Basic Three-Layer NN

In [6]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

w1 = weight_variable('w1', [784,50])
b1 = bias_variable('b1', [50])

z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.relu(z1)

w2 = weight_variable('w2', [50,25])
b2 = bias_variable('b2', [25])

z2 = tf.matmul(a1, w2) + b2
a2 = tf.nn.relu(z2)

w3 = weight_variable('w3', [25,10])
b3 = bias_variable('b3', [10])

z3 = tf.matmul(a2, w3) + b3

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = z3))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(z3,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(1000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))

step 0, train accuracy 0.07
step 100, train accuracy 0.91
step 200, train accuracy 0.91
step 300, train accuracy 0.95
step 400, train accuracy 0.87
step 500, train accuracy 0.94
step 600, train accuracy 0.91
step 700, train accuracy 0.89
step 800, train accuracy 0.96
step 900, train accuracy 0.92
test accuracy 0.9523


### Basic Three-Layer NN with BatchNorm

In [7]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])
phase = tf.placeholder(tf.bool, name='phase')

w1 = weight_variable('w1', [784,50])
b1 = bias_variable('b1', [50])

z1 = tf.matmul(x, w1) + b1
zbn1 = tf.layers.batch_normalization(z1, training = phase)
a1 = tf.nn.relu(zbn1)

w2 = weight_variable('w2', [50, 25])
b2 = bias_variable('b2', [25])

z2 = tf.matmul(a1, w2) + b2
zbn2 = tf.layers.batch_normalization(z2, training = phase)
a2 = tf.nn.relu(zbn2)

w3 = weight_variable('w3', [25, 10])
b3 = bias_variable('b3', [10])

z3 = tf.matmul(a2, w3) + b3
zbn3 = tf.layers.batch_normalization(z3, training = phase)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = zbn3))

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(zbn3,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(2000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], phase:True})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run([train_step, update_ops], feed_dict={x: batch[0], y_: batch[1], phase: True})
    print("test accuracy %s"% (accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, phase: False})))

step 0, train accuracy 0.07
step 100, train accuracy 0.85
step 200, train accuracy 0.9
step 300, train accuracy 0.98
step 400, train accuracy 0.95
step 500, train accuracy 0.92
step 600, train accuracy 0.93
step 700, train accuracy 0.92
step 800, train accuracy 0.97
step 900, train accuracy 0.94
step 1000, train accuracy 0.93
step 1100, train accuracy 0.97
step 1200, train accuracy 0.96
step 1300, train accuracy 0.95
step 1400, train accuracy 0.94
step 1500, train accuracy 0.94
step 1600, train accuracy 0.93
step 1700, train accuracy 0.94
step 1800, train accuracy 0.95
step 1900, train accuracy 0.92
test accuracy 0.969


### Basic Three-Layer NN with BatchNorm and Dropout

In [8]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])
phase = tf.placeholder(tf.bool, name='phase')
keep_prob = tf.placeholder(tf.float32)

w1 = weight_variable('w1', [784,50])
b1 = bias_variable('b1', [50])

z1 = tf.matmul(x, w1) + b1
zbn1 = tf.layers.batch_normalization(z1, training = phase)
a1 = tf.nn.relu(zbn1)

w2 = weight_variable('w2', [50, 25])
b2 = bias_variable('b2', [25])

z2 = tf.matmul(a1, w2) + b2
zbn2 = tf.layers.batch_normalization(z2, training = phase)
a2 = tf.nn.relu(zbn2)
h_fc2_drop = tf.nn.dropout(a2, keep_prob)


w3 = weight_variable('w3', [25, 10])
b3 = bias_variable('b3', [10])

z3 = tf.matmul(h_fc2_drop, w3) + b3
zbn3 = tf.layers.batch_normalization(z3, training = phase)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = zbn3))

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
    
correct_prediction = tf.equal(tf.argmax(zbn3,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(5000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], phase:True, keep_prob: 0.5})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run([train_step, update_ops], feed_dict={x: batch[0], y_: batch[1], phase: True, keep_prob: 0.5})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, phase: False, keep_prob: 1.0})))

step 0, train accuracy 0.14
step 100, train accuracy 0.79
step 200, train accuracy 0.82
step 300, train accuracy 0.86
step 400, train accuracy 0.89
step 500, train accuracy 0.9
step 600, train accuracy 0.85
step 700, train accuracy 0.88
step 800, train accuracy 0.84
step 900, train accuracy 0.9
step 1000, train accuracy 0.9
step 1100, train accuracy 0.86
step 1200, train accuracy 0.86
step 1300, train accuracy 0.92
step 1400, train accuracy 0.91
step 1500, train accuracy 0.91
step 1600, train accuracy 0.89
step 1700, train accuracy 0.92
step 1800, train accuracy 0.88
step 1900, train accuracy 0.89
step 2000, train accuracy 0.89
step 2100, train accuracy 0.91
step 2200, train accuracy 0.86
step 2300, train accuracy 0.88
step 2400, train accuracy 0.94
step 2500, train accuracy 0.89
step 2600, train accuracy 0.92
step 2700, train accuracy 0.9
step 2800, train accuracy 0.95
step 2900, train accuracy 0.97
step 3000, train accuracy 0.91
step 3100, train accuracy 0.86
step 3200, train accurac

### Basic Four-Layer NN

In [9]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

w1 = weight_variable('w1', [784,128])
b1 = bias_variable('b1', [128])

z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.relu(z1)

w2 = weight_variable('w2', [128, 64])
b2 = bias_variable('b2', [64])

z2 = tf.matmul(a1, w2) + b2
a2 = tf.nn.relu(z2)

w3 = weight_variable('w3', [64, 32])
b3 = bias_variable('b3', [32])

z3 = tf.matmul(a2, w3) + b3
a3 = tf.nn.relu(z3)

w4 = weight_variable('w4', [32, 10])
b4 = bias_variable('b4', [10])

z4 = tf.matmul(a3, w4) + b4

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = z4))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(z4,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(1000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))

step 0, train accuracy 0.09
step 100, train accuracy 0.96
step 200, train accuracy 0.94
step 300, train accuracy 0.89
step 400, train accuracy 1
step 500, train accuracy 0.96
step 600, train accuracy 0.97
step 700, train accuracy 0.92
step 800, train accuracy 0.99
step 900, train accuracy 0.97
test accuracy 0.9588


### Basic Four-Layer NN with Dropout in Last Layer

In [10]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])
keep_prob = tf.placeholder(tf.float32)

w1 = weight_variable('w1', [784,128])
b1 = bias_variable('b1', [128])

z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.relu(z1)

w2 = weight_variable('w2', [128, 64])
b2 = bias_variable('b2', [64])

z2 = tf.matmul(a1, w2) + b2
a2 = tf.nn.relu(z2)

w3 = weight_variable('w3', [64, 32])
b3 = bias_variable('b3', [32])

z3 = tf.matmul(a2, w3) + b3
a3 = tf.nn.relu(z3)
h_fc3_drop = tf.nn.dropout(a3, keep_prob)

w4 = weight_variable('w4', [32, 10])
b4 = bias_variable('b4', [10])

z4 = tf.matmul(h_fc3_drop, w4) + b4


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = z4))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(z4,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(5000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 0.5})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

step 0, train accuracy 0.1
step 100, train accuracy 0.77
step 200, train accuracy 0.8
step 300, train accuracy 0.89
step 400, train accuracy 0.89
step 500, train accuracy 0.89
step 600, train accuracy 0.93
step 700, train accuracy 0.94
step 800, train accuracy 0.95
step 900, train accuracy 0.94
step 1000, train accuracy 0.86
step 1100, train accuracy 0.97
step 1200, train accuracy 0.92
step 1300, train accuracy 0.97
step 1400, train accuracy 0.94
step 1500, train accuracy 0.94
step 1600, train accuracy 0.92
step 1700, train accuracy 0.95
step 1800, train accuracy 0.94
step 1900, train accuracy 0.95
step 2000, train accuracy 0.96
step 2100, train accuracy 0.96
step 2200, train accuracy 0.93
step 2300, train accuracy 0.96
step 2400, train accuracy 0.94
step 2500, train accuracy 0.92
step 2600, train accuracy 0.95
step 2700, train accuracy 0.96
step 2800, train accuracy 0.98
step 2900, train accuracy 0.96
step 3000, train accuracy 0.99
step 3100, train accuracy 0.96
step 3200, train accur

### Basic Four-Layer NN with Dropout in All Layers

In [11]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])
keep_prob = tf.placeholder(tf.float32)

w1 = weight_variable('w1', [784,128])
b1 = bias_variable('b1', [128])

z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.relu(z1)
h_fc1_drop = tf.nn.dropout(a1, keep_prob)

w2 = weight_variable('w2', [128, 64])
b2 = bias_variable('b2', [64])

z2 = tf.matmul(h_fc1_drop, w2) + b2
a2 = tf.nn.relu(z2)
h_fc2_drop = tf.nn.dropout(a2, keep_prob)

w3 = weight_variable('w3', [64, 32])
b3 = bias_variable('b3', [32])

z3 = tf.matmul(h_fc2_drop, w3) + b3
a3 = tf.nn.relu(z3)
h_fc3_drop = tf.nn.dropout(a3, keep_prob)

w4 = weight_variable('w4', [32, 10])
b4 = bias_variable('b4', [10])

z4 = tf.matmul(h_fc3_drop, w4) + b4


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = z4))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(z4,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(3000):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 0.5})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

step 0, train accuracy 0.13
step 100, train accuracy 0.62
step 200, train accuracy 0.65
step 300, train accuracy 0.69
step 400, train accuracy 0.8
step 500, train accuracy 0.75
step 600, train accuracy 0.73
step 700, train accuracy 0.83
step 800, train accuracy 0.76
step 900, train accuracy 0.82
step 1000, train accuracy 0.87
step 1100, train accuracy 0.79
step 1200, train accuracy 0.75
step 1300, train accuracy 0.8
step 1400, train accuracy 0.8
step 1500, train accuracy 0.86
step 1600, train accuracy 0.79
step 1700, train accuracy 0.86
step 1800, train accuracy 0.83
step 1900, train accuracy 0.82
step 2000, train accuracy 0.81
step 2100, train accuracy 0.87
step 2200, train accuracy 0.87
step 2300, train accuracy 0.82
step 2400, train accuracy 0.81
step 2500, train accuracy 0.78
step 2600, train accuracy 0.78
step 2700, train accuracy 0.77
step 2800, train accuracy 0.83
step 2900, train accuracy 0.82
test accuracy 0.9261


## Convolutional Neural Nets

### ConvNet

In [12]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])
keep_prob = tf.placeholder(tf.float32)

W_conv1 = weight_variable('W_conv1', [5, 5, 1, 32])
b_conv1 = bias_variable('b_conv1', [32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(tf.nn.conv2d(input = x_image, filter = W_conv1, strides = [1,1,1,1], padding = 'SAME') + b_conv1)
h_pool1 = tf.nn.max_pool(value = h_conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W_conv2 = weight_variable('W_conv2', [5, 5, 32, 64])
b_conv2 = bias_variable('b_conv2', [64])

h_conv2 = tf.nn.relu(tf.nn.conv2d(input = h_pool1, filter = W_conv2, strides = [1,1,1,1], padding = 'SAME') + b_conv2)
h_pool2 = tf.nn.max_pool(value = h_conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W_fc1 = weight_variable('W_fc1', [7 * 7 * 64, 1024])
b_fc1 = bias_variable('b_fc1', [1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable('W_fc2', [1024, 10])
b_fc2 = bias_variable('b_fc2', [10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(300):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 0.5})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

step 0, train accuracy 0.12
step 100, train accuracy 0.94
step 200, train accuracy 0.99
test accuracy 0.9853


### ConvNet 2

In [13]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])
keep_prob = tf.placeholder(tf.float32)

W_conv1 = weight_variable('W_conv1', [3, 3, 1, 32])
b_conv1 = bias_variable('b_conv1', [32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(tf.nn.conv2d(input = x_image, filter = W_conv1, strides = [1,1,1,1], padding = 'SAME') + b_conv1)
h_pool1 = tf.nn.max_pool(value = h_conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W_conv2 = weight_variable('W_conv2', [5, 5, 32, 64])
b_conv2 = bias_variable('b_conv2', [64])

h_conv2 = tf.nn.relu(tf.nn.conv2d(input = h_pool1, filter = W_conv2, strides = [1,1,1,1], padding = 'SAME') + b_conv2)
h_pool2 = tf.nn.max_pool(value = h_conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W_fc1 = weight_variable('W_fc1', [7 * 7 * 64, 1024])
b_fc1 = bias_variable('b_fc1', [1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable('W_fc2', [1024, 10])
b_fc2 = bias_variable('b_fc2', [10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(300):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 0.5})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

step 0, train accuracy 0.17
step 100, train accuracy 0.96
step 200, train accuracy 0.97
test accuracy 0.9803


### ResNet

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

W_conv1 = weight_variable('W_conv1', [3, 3, 1, 32])
b_conv1 = bias_variable('b_conv1', [32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(tf.nn.conv2d(input = x_image, filter = W_conv1, strides = [1,1,1,1], padding = 'SAME') + b_conv1)
h_pool1 = tf.nn.max_pool(value = h_conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W_conv2 = weight_variable('W_conv2', [5, 5, 32, 64])
b_conv2 = bias_variable('b_conv2', [64])

h_conv2 = tf.nn.relu(tf.nn.conv2d(input = h_pool1, filter = W_conv2, strides = [1,1,1,1], padding = 'SAME') + b_conv2)
# h_pool2 = tf.nn.max_pool(value = h_conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W_skconv2 = weight_variable('W_skconv2', [2, 2, 1, 64])
b_skconv2 = bias_variable('b_skconv2', [64])

x_skip1 = tf.nn.conv2d(input = x_image, filter = W_skconv2, strides = [1,2,2,1], padding = 'VALID') + b_skconv2

W_conv3 = weight_variable('W_conv3', [3, 3, 64, 128])
b_conv3 = bias_variable('b_conv3', [128])

h_conv2_add = tf.add(h_conv2, x_skip1)

h_conv3 = tf.nn.relu(tf.nn.conv2d(input = h_conv2_add, filter = W_conv3, strides = [1,1,1,1], padding = 'SAME') + b_conv3)
h_pool3 = tf.nn.max_pool(value = h_conv3, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W_fc1 = weight_variable('W_fc1', [7 * 7 * 128, 1024])
b_fc1 = bias_variable('b_fc1', [1024])

h_pool3_flat = tf.reshape(h_pool3, [-1, 7*7*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable('W_fc2', [1024, 10])
b_fc2 = bias_variable('b_fc2', [10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(500):
        batch = mnist.train.next_batch(100)
        if _%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 0.5})
            print("step %d, train accuracy %g"%(_, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print("test accuracy %s"% (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

step 0, train accuracy 0.12
step 100, train accuracy 0.98
step 200, train accuracy 0.99
step 300, train accuracy 0.98
